In [1]:
import numpy as np
import argparse
import os
import imp
import re
import pickle
import datetime
import random
import math
import copy


import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils import data
from torch.autograd import Variable
import torch.nn.functional as F


from utils import utils
from utils.readers import InvasiveVentilationReader
from utils.preprocessing import Discretizer, Normalizer
from utils import metrics
from utils import common_utils

### Prepare

In [2]:
data_path = './data/'
file_name = './model/concare0'
small_part = False
arg_timestep = 1.0
batch_size = 256
epochs = 100

In [3]:
train_reader = InvasiveVentilationReader(dataset_dir=os.path.join(data_path, 'train'),
                                         listfile=os.path.join(data_path, 'train_listfile.csv'))

In [11]:
path_ = os.path.join(data_path, 'train')
e_ts = '14978979_episode1_timeseries.csv'

path_file = os.path.join(path_, e_ts)
print(path_file)

import pandas as pd
e_ts = pd.read_csv(path_file)
print(e_ts.columns)

./data/train/14978979_episode1_timeseries.csv
Index(['Hours', 'Albumin', 'Anion gap', 'Arterial O2 pressure', 'Bicarbonate',
       'Bilirubin', 'Blood urea nitrogen', 'CO2 (ETCO2, PCO2, etc.)',
       'Calcium', 'Calcium ionized', 'Cardiac Index',
       'Central Venous Pressure', 'Chloride', 'Creatinine',
       'Diastolic blood pressure', 'Expiratory Ratio',
       'Fraction inspired oxygen', 'Glascow coma scale eye opening',
       'Glascow coma scale motor response',
       'Glascow coma scale verbal response', 'Glucose', 'Heart Rate', 'Height',
       'Hematocrit', 'Hemoglobin', 'Inspiratory Ratio', 'Lactate',
       'Leukocytes', 'Magnesium', 'Mean blood pressure', 'Minute Volume',
       'Negative Insp. Force', 'Oxygen saturation',
       'Partial pressure of carbon dioxide', 'Partial thromboplastin time',
       'Plateau Pressure', 'Platelets', 'Positive end-expiratory pressure',
       'Potassium', 'Prothrombin time', 'Respiratory rate', 'SOFA', 'Sodium',
       'Systolic blo

In [4]:
# Build readers, discretizers, normalizers
train_reader = InvasiveVentilationReader(dataset_dir=os.path.join(data_path, 'train'),
                                         listfile=os.path.join(data_path, 'train_listfile.csv'))

val_reader = InvasiveVentilationReader(dataset_dir=os.path.join(data_path, 'train'),
                                       listfile=os.path.join(data_path, 'val_listfile.csv'))

discretizer = Discretizer(timestep=arg_timestep,
                          store_masks=True,
                          impute_strategy='previous',
                          start_time='zero')

os.path.dirname(__file__) /Users/xutian/Documents/Dev/dartmouth/research/IV/utils
__File__ /Users/xutian/Documents/Dev/dartmouth/research/IV/utils/preprocessing.py
self._header ['Hours', 'Capillary refill rate', 'Diastolic blood pressure', 'Fraction inspired oxygen', 'Glascow coma scale eye opening', 'Glascow coma scale motor response', 'Glascow coma scale total', 'Glascow coma scale verbal response', 'Glucose', 'Heart Rate', 'Height', 'Mean blood pressure', 'Oxygen saturation', 'Respiratory rate', 'Systolic blood pressure', 'Temperature', 'Weight', 'pH']
self._timestep 1.0
self._store_masks True
self._id_to_channel ['Capillary refill rate', 'Diastolic blood pressure', 'Fraction inspired oxygen', 'Glascow coma scale eye opening', 'Glascow coma scale motor response', 'Glascow coma scale total', 'Glascow coma scale verbal response', 'Glucose', 'Heart Rate', 'Height', 'Mean blood pressure', 'Oxygen saturation', 'Respiratory rate', 'Systolic blood pressure', 'Temperature', 'Weight', 'pH']


In [4]:
discretizer_header = discretizer.transform(train_reader.read_example(0)["X"])[1].split(',')
cont_channels = [i for (i, x) in enumerate(discretizer_header) if x.find("->") == -1]

normalizer = Normalizer(fields=cont_channels)  # choose here which columns to standardize
normalizer_state = 'ihm_normalizer'
normalizer_state = os.path.join(os.path.dirname(data_path), normalizer_state)
normalizer.load_params(normalizer_state)

In [5]:
n_trained_chunks = 0
train_raw = utils.load_data(train_reader, discretizer, normalizer, small_part, return_names=True)
val_raw = utils.load_data(val_reader, discretizer, normalizer, small_part, return_names=True)

In [6]:
demographic_data = []
diagnosis_data = []
idx_list = []

demo_path = data_path + 'demographic/'
print("demo_path: ", demo_path)

demo_path:  ./data/demographic/


In [7]:
# cnt = 0
for cur_name in os.listdir(demo_path):
#     if cnt >= 2:
#         continue
#     cnt += 1
    cur_id, cur_episode = cur_name.split('_', 1)
#     print("cur_episode: ", cur_episode)
    cur_episode = cur_episode[:-4]
    cur_file = demo_path + cur_name
    
#     print('cur_id: ', cur_id, "cur_episode: ", cur_episode, "cur_file: ", cur_file)

    with open(cur_file, "r") as tsfile:
        header = tsfile.readline().strip().split(',')
        if header[0] != "Icustay":
            continue
        cur_data = tsfile.readline().strip().split(',')
    
#     print("cur_data", cur_data)
    if len(cur_data) == 1:
        cur_demo = np.zeros(12)
        cur_diag = np.zeros(128)
    else:
        if cur_data[3] == '':
            cur_data[3] = 60.0
        if cur_data[4] == '':
            cur_data[4] = 160
        if cur_data[5] == '':
            cur_data[5] = 60

        cur_demo = np.zeros(12)
        cur_demo[int(cur_data[1])] = 1
        cur_demo[5 + int(cur_data[2])] = 1
        cur_demo[9:] = cur_data[3:6]
        cur_diag = np.array(cur_data[8:], dtype=np.int)

    demographic_data.append(cur_demo)
    diagnosis_data.append(cur_diag)
    idx_list.append(cur_id+'_'+cur_episode)

for each_idx in range(9,12):
    cur_val = []
    for i in range(len(demographic_data)):
        cur_val.append(demographic_data[i][each_idx])
    cur_val = np.array(cur_val)
    _mean = np.mean(cur_val)
    _std = np.std(cur_val)
    _std = _std if _std > 1e-7 else 1e-7
    for i in range(len(demographic_data)):
        demographic_data[i][each_idx] = (demographic_data[i][each_idx] - _mean) / _std

/Users/xutian/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() == True else 'cpu')
#device = torch.device('cpu')
print("available device: {}".format(device))

available device: cpu


### model

In [9]:
import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils import data
from torch.autograd import Variable
import torch.nn.functional as F

RANDOM_SEED = 12345
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic=True

# TODO: LSTM Model 
# maybe this can help
# https://github.com/spdin/time-series-prediction-lstm-pytorch/blob/master/Time_Series_Prediction_with_LSTM_Using_PyTorch.ipynb    
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out



In [10]:
def get_loss(y_pred, y_true):
    loss = torch.nn.BCELoss()
    return loss(y_pred, y_true)

In [ ]:
# LSTM
model = LSTM(num_classes=1, input_size=78, hidden_size=32, num_layers=1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [11]:
class Dataset(data.Dataset):
    def __init__(self, x, y, name):
        self.x = x
        self.y = y
        self.name = name

    def __getitem__(self, index):#返回的是tensor
        return self.x[index], self.y[index], self.name[index]

    def __len__(self):
        return len(self.x)

In [12]:
train_dataset = Dataset(train_raw['data'][0], train_raw['data'][1], train_raw['names'])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataset = Dataset(val_raw['data'][0], val_raw['data'][1], val_raw['names'])
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [13]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED) #numpy
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED) # cpu
torch.cuda.manual_seed(RANDOM_SEED) #gpu
torch.backends.cudnn.deterministic=True # cudnn

model = ConCare(input_dim = 76, hidden_dim = 64, d_model = 64,  MHD_num_head = 4 , d_ff = 256, output_dim = 1).to(device)
# input_dim, d_model, d_k, d_v, MHD_num_head, d_ff, output_dim
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

max_roc = 0
max_prc = 0
train_loss = []
train_model_loss = []
train_decov_loss = []
valid_loss = []
valid_model_loss = []
valid_decov_loss = []
history = []
np.set_printoptions(threshold=np.inf)
np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)

### Run

In [15]:
max_roc = 0
max_prc = 0
min_mad = 100

train_loss = []
train_model_loss = []
train_decov_loss = []
valid_loss = []
valid_model_loss = []
valid_decov_loss = []
history = []
np.set_printoptions(threshold=np.inf)
np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)

for each_epoch in range(epochs*2):
    batch_loss = []
    model_batch_loss = []
    decov_batch_loss = []

    model.train()
 
    for step, (batch_x, batch_y, batch_name) in enumerate(train_loader):   
        optimizer.zero_grad()
        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float().to(device)
        #print(">>>> batch_y: ", batch_y)
        batch_demo = []
        for i in range(len(batch_name)):
            cur_id, cur_ep, _ = batch_name[i].split('_', 2)
            cur_idx = cur_id + '_' + cur_ep
#             print("cur_idx", cur_idx)
#             print("demographic_data", demographic_data)
#             print("idx_list: ", idx_list)
            cur_demo = torch.tensor(demographic_data[idx_list.index(cur_idx)], dtype=torch.float32)
            batch_demo.append(cur_demo)
        
        batch_demo = torch.stack(batch_demo).to(device)
        output, decov_loss = model(batch_x, batch_demo)
        
        #print("batch_y.unsqueeze(-1): ", batch_y.unsqueeze(-1).shape)
        #print("output.shape: ", output.shape)
        model_loss = get_loss(output, batch_y.unsqueeze(-1))
        loss = model_loss + 1000* decov_loss
        
        batch_loss.append(loss.cpu().detach().numpy())
        model_batch_loss.append(model_loss.cpu().detach().numpy())
        decov_batch_loss.append(decov_loss.cpu().detach().numpy())
        loss.backward()
        optimizer.step()
        
        if step % 30 == 0:
            print('Epoch %d Batch %d: Train Loss = %.4f'%(each_epoch, step, np.mean(np.array(batch_loss))))
            print('Model Loss = %.4f, Decov Loss = %.4f'%(np.mean(np.array(model_batch_loss)), np.mean(np.array(decov_batch_loss))))
    train_loss.append(np.mean(np.array(batch_loss)))
    train_model_loss.append(np.mean(np.array(model_batch_loss)))
    train_decov_loss.append(np.mean(np.array(decov_batch_loss)))
    
    batch_loss = []
    model_batch_loss = []
    decov_batch_loss = []
    
    y_true = []
    y_pred = []
    with torch.no_grad():
        model.eval()
        for step, (batch_x, batch_y, batch_name) in enumerate(valid_loader):
            batch_x = batch_x.float().to(device)
            batch_y = batch_y.float().to(device)
            batch_demo = []
            for i in range(len(batch_name)):
                cur_id, cur_ep, _ = batch_name[i].split('_', 2)
                cur_idx = cur_id + '_' + cur_ep
                cur_demo = torch.tensor(demographic_data[idx_list.index(cur_idx)], dtype=torch.float32)
                batch_demo.append(cur_demo)

            batch_demo = torch.stack(batch_demo).to(device)
            output,decov_loss = model(batch_x, batch_demo)
            
            model_loss = get_loss(output, batch_y.unsqueeze(-1))

            loss = model_loss + 1000* decov_loss
            batch_loss.append(loss.cpu().detach().numpy())
            model_batch_loss.append(model_loss.cpu().detach().numpy())
            decov_batch_loss.append(decov_loss.cpu().detach().numpy())
            y_pred += list(output.cpu().detach().numpy().flatten())
            y_true += list(batch_y.cpu().numpy().flatten())
            
    valid_loss.append(np.mean(np.array(batch_loss)))
    valid_model_loss.append(np.mean(np.array(model_batch_loss)))
    valid_decov_loss.append(np.mean(np.array(decov_batch_loss)))
    
    print("\n==>Predicting on validation")
    print('Valid Loss = %.4f'%(valid_loss[-1]))
    print('valid_model Loss = %.4f'%(valid_model_loss[-1]))
    print('valid_decov Loss = %.4f'%(valid_decov_loss[-1]))
    y_pred = np.array(y_pred)
 
    ret = metrics.print_metrics_regression(y_true, y_pred)
    history.append(ret)
    print()

#     cur_auroc = ret['auroc']
    cur_mad = ret['mad']
    if cur_mad < min_mad:
        min_mad = cur_mad
        state = {
            'net': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': each_epoch
        }
        torch.save(state, file_name)
        print('\n------------ Save best model ------------\n')
   

step:  0
batch_x: 
 tensor([[[ 1.0000,  0.0000, -0.0387,  ...,  0.0000,  1.0000,  1.0000],
         [ 1.0000,  0.0000, -0.0435,  ...,  0.0000,  0.0000,  1.0000],
         [ 1.0000,  0.0000, -0.0150,  ...,  1.0000,  0.0000,  0.0000],
         ...,
         [ 1.0000,  0.0000, -0.0245,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.0000,  0.0000, -0.0245,  ...,  1.0000,  0.0000,  0.0000],
         [ 1.0000,  0.0000, -0.0340,  ...,  0.0000,  0.0000,  1.0000]],

        [[ 1.0000,  0.0000, -0.0198,  ...,  1.0000,  0.0000,  0.0000],
         [ 1.0000,  0.0000, -0.0292,  ...,  1.0000,  0.0000,  1.0000],
         [ 1.0000,  0.0000,  0.0181,  ...,  1.0000,  0.0000,  1.0000],
         ...,
         [ 1.0000,  0.0000, -0.1098,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.0000,  0.0000, -0.1192,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.0000,  0.0000, -0.0956,  ...,  1.0000,  0.0000,  0.0000]],

        [[ 1.0000,  0.0000,  0.0134,  ...,  1.0000,  1.0000,  0.0000],
         [ 1.0000,  0.000

/Users/xutian/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:140: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/xutian/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:208: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 0 Batch 0: Train Loss = 0.4148
Model Loss = 0.4102, Decov Loss = 0.0000
step:  1
batch_x: 
 tensor([[[ 1.0000e+00,  0.0000e+00, -1.0302e-02,  ...,  0.0000e+00,
           0.0000e+00,  1.0000e+00],
         [ 1.0000e+00,  0.0000e+00, -4.3460e-02,  ...,  1.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.0000e+00,  0.0000e+00,  7.0225e-02,  ...,  1.0000e+00,
           0.0000e+00,  1.0000e+00],
         ...,
         [ 1.0000e+00,  0.0000e+00, -8.2776e-04,  ...,  1.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.0000e+00,  0.0000e+00, -5.7670e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.0000e+00,  0.0000e+00, -4.8197e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[ 1.0000e+00,  0.0000e+00, -1.0302e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.0000e+00,  0.0000e+00, -5.2933e-02,  ...,  1.0000e+00,
           1.0000e+00,  0.0000e+00],
         [ 1.0000e+00,  0.0000e+00, -1.0978e-0

step:  2
batch_x: 
 tensor([[[ 1.0000e+00,  0.0000e+00, -1.0302e-02,  ...,  0.0000e+00,
           1.0000e+00,  1.0000e+00],
         [ 1.0000e+00,  0.0000e+00, -1.0302e-02,  ...,  0.0000e+00,
           1.0000e+00,  1.0000e+00],
         [ 1.0000e+00,  0.0000e+00, -1.0302e-02,  ...,  0.0000e+00,
           0.0000e+00,  1.0000e+00],
         ...,
         [ 1.0000e+00,  0.0000e+00,  2.7593e-02,  ...,  0.0000e+00,
           0.0000e+00,  1.0000e+00],
         [ 1.0000e+00,  0.0000e+00, -2.4512e-02,  ...,  0.0000e+00,
           0.0000e+00,  1.0000e+00],
         [ 1.0000e+00,  0.0000e+00, -1.9775e-02,  ...,  0.0000e+00,
           0.0000e+00,  1.0000e+00]],

        [[ 1.0000e+00,  0.0000e+00,  7.4962e-02,  ...,  1.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.0000e+00,  0.0000e+00,  4.1804e-02,  ...,  0.0000e+00,
           0.0000e+00,  1.0000e+00],
         [ 1.0000e+00,  0.0000e+00,  8.4436e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
     

step:  3
batch_x: 
 tensor([[[ 1.0000e+00,  0.0000e+00, -1.0302e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.0000e+00,  0.0000e+00,  1.6970e-01,  ...,  1.0000e+00,
           1.0000e+00,  0.0000e+00],
         [ 1.0000e+00,  0.0000e+00,  1.2707e-01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [ 1.0000e+00,  0.0000e+00,  4.1804e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.0000e+00,  0.0000e+00,  1.8120e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.0000e+00,  0.0000e+00,  6.0752e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[ 1.0000e+00,  0.0000e+00, -1.0302e-02,  ...,  0.0000e+00,
           1.0000e+00,  1.0000e+00],
         [ 1.0000e+00,  0.0000e+00, -1.0302e-02,  ...,  0.0000e+00,
           1.0000e+00,  1.0000e+00],
         [ 1.0000e+00,  0.0000e+00, -1.0302e-02,  ...,  0.0000e+00,
           1.0000e+00,  1.0000e+00],
         ...,
     

...

step:  54
batch_x: 
 tensor([[[ 1.0000,  0.0000, -0.0103,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.0000,  0.0000, -0.0103,  ...,  0.0000,  0.0000,  1.0000],
         [ 1.0000,  0.0000, -0.0103,  ...,  0.0000,  0.0000,  1.0000],
         ...,
         [ 1.0000,  0.0000, -0.0292,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.0000,  0.0000, -0.0292,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.0000,  0.0000, -0.0150,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 1.0000,  0.0000, -0.0103,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.0000,  0.0000, -0.0103,  ...,  0.0000,  0.0000,  1.0000],
         [ 1.0000,  0.0000, -0.0103,  ...,  0.0000,  0.0000,  1.0000],
         ...,
         [ 1.0000,  0.0000,  0.0750,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.0000,  0.0000,  0.1034,  ...,  1.0000,  0.0000,  0.0000],
         [ 1.0000,  0.0000,  0.0939,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 1.0000,  0.0000,  0.0276,  ...,  1.0000,  1.0000,  0.0000],
         [ 1.0000,  0.00

step:  55
batch_x: 
 tensor([[[ 1.0000e+00,  0.0000e+00, -1.0302e-02,  ...,  0.0000e+00,
           0.0000e+00,  1.0000e+00],
         [ 1.0000e+00,  0.0000e+00, -5.7670e-02,  ...,  1.0000e+00,
           1.0000e+00,  1.0000e+00],
         [ 1.0000e+00,  0.0000e+00,  2.2857e-02,  ...,  1.0000e+00,
           0.0000e+00,  1.0000e+00],
         ...,
         [ 1.0000e+00,  0.0000e+00, -4.3460e-02,  ...,  1.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.0000e+00,  0.0000e+00, -2.4512e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.0000e+00,  0.0000e+00, -5.2933e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[ 1.0000e+00,  0.0000e+00,  1.8120e-02,  ...,  0.0000e+00,
           0.0000e+00,  1.0000e+00],
         [ 1.0000e+00,  0.0000e+00, -2.9249e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.0000e+00,  0.0000e+00, -4.8197e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
    

step:  56
batch_x: 
 tensor([[[ 1.0000e+00,  0.0000e+00, -3.8723e-02,  ...,  1.0000e+00,
           1.0000e+00,  0.0000e+00],
         [ 1.0000e+00,  0.0000e+00, -1.9775e-02,  ...,  0.0000e+00,
           1.0000e+00,  0.0000e+00],
         [ 1.0000e+00,  0.0000e+00, -3.8723e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [ 1.0000e+00,  0.0000e+00, -1.5038e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.0000e+00,  0.0000e+00,  1.6970e-01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.0000e+00,  0.0000e+00,  8.4436e-02,  ...,  0.0000e+00,
           0.0000e+00,  1.0000e+00]],

        [[ 1.0000e+00,  0.0000e+00, -1.0302e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.0000e+00,  0.0000e+00, -1.0302e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.0000e+00,  0.0000e+00, -8.1355e-02,  ...,  1.0000e+00,
           1.0000e+00,  0.0000e+00],
         ...,
    

step:  57
batch_x: 
 tensor([[[ 1.0000,  0.0000, -0.0103,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.0000,  0.0000, -0.0103,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.0000,  0.0000, -0.0103,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 1.0000,  0.0000, -0.0292,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.0000,  0.0000, -0.0719,  ...,  1.0000,  0.0000,  0.0000],
         [ 1.0000,  0.0000, -0.0956,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 1.0000,  0.0000, -0.0103,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.0000,  0.0000, -0.0198,  ...,  1.0000,  1.0000,  0.0000],
         [ 1.0000,  0.0000, -0.0245,  ...,  1.0000,  0.0000,  0.0000],
         ...,
         [ 1.0000,  0.0000,  0.0371,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.0000,  0.0000,  0.0892,  ...,  1.0000,  0.0000,  0.0000],
         [ 1.0000,  0.0000,  0.0797,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 1.0000,  0.0000, -0.0103,  ...,  0.0000,  0.0000,  1.0000],
         [ 1.0000,  0.00

FileNotFoundError: [Errno 2] No such file or directory: './model/concare0'

### Run for test

In [13]:
checkpoint = torch.load(file_name)
save_epoch = checkpoint['epoch']
model.load_state_dict(checkpoint['net'])
optimizer.load_state_dict(checkpoint['optimizer'])
model.eval()

test_reader = VentilationDuratingReader(dataset_dir=os.path.join(data_path, 'test'),
                                            listfile=os.path.join(data_path, 'test_listfile.csv'),
                                            period_length=48.0)
test_raw = utils.load_data(test_reader, discretizer, normalizer, small_part, return_names=True)
# print(test_raw)
test_dataset = Dataset(test_raw['data'][0], test_raw['data'][1], test_raw['names'])
# print(">>", test_dataset)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print("test_loader: ", test_loader)

In [14]:
batch_loss = []
y_true = []
y_pred = []
with torch.no_grad():
    model.eval()
    print("test_loader: ", test_loader)
    for step, (batch_x, batch_y, batch_name) in enumerate(test_loader):
        
        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float().to(device)
        batch_demo = []
        for i in range(len(batch_name)):
            cur_id, cur_ep, _ = batch_name[i].split('_', 2)
            cur_idx = cur_id + '_' + cur_ep
            cur_demo = torch.tensor(demographic_data[idx_list.index(cur_idx)], dtype=torch.float32)
            batch_demo.append(cur_demo)

        batch_demo = torch.stack(batch_demo).to(device)
        output = model(batch_x, batch_demo)[0]

        loss = get_loss(output, batch_y.unsqueeze(-1))
        batch_loss.append(loss.cpu().detach().numpy())
        y_pred += list(output.cpu().detach().numpy().flatten())
        y_true += list(batch_y.cpu().numpy().flatten())

print("\n==>Predicting on test")
print('Test Loss = %.4f'%(np.mean(np.array(batch_loss))))
y_pred = np.array(y_pred)
# y_pred = np.stack([1 - y_pred, y_pred], axis=1)
test_res = metrics.print_metrics_regression(y_true, y_pred)

/home/ubicomp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:140: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ubicomp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:208: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.



==>Predicting on test
Test Loss = 0.2532
confusion matrix:
[[2811   51]
 [ 270  104]]
accuracy = 0.9008034467697144
precision class 0 = 0.9123660922050476
precision class 1 = 0.6709677577018738
recall class 0 = 0.9821802973747253
recall class 1 = 0.27807486057281494
AUC of ROC = 0.8719613822277529
AUC of PRC = 0.5344983146118463
min(+P, Se) = 0.5106951871657754
f1_score = 0.3931947039659704


In [15]:
# Bootstrap
N = len(y_true)
N_idx = np.arange(N)
K = 1000

auroc = []
auprc = []
minpse = []
for i in range(K):
    boot_idx = np.random.choice(N_idx, N, replace=True)
    boot_true = np.array(y_true)[boot_idx]
    boot_pred = y_pred[boot_idx, :]
    test_ret = metrics.print_metrics_binary(boot_true, boot_pred, verbose=0)
    auroc.append(test_ret['auroc'])
    auprc.append(test_ret['auprc'])
    minpse.append(test_ret['minpse'])
    print('%d/%d'%(i+1,K))
    
print('auroc %.4f(%.4f)'%(np.mean(auroc), np.std(auroc)))
print('auprc %.4f(%.4f)'%(np.mean(auprc), np.std(auprc)))
print('minpse %.4f(%.4f)'%(np.mean(minpse), np.std(minpse)))

1/1000
2/1000
3/1000
4/1000
5/1000
6/1000
7/1000
8/1000
9/1000
10/1000
11/1000
12/1000
13/1000
14/1000
15/1000
16/1000
17/1000
18/1000
19/1000
20/1000
21/1000
22/1000
23/1000
24/1000
25/1000
26/1000
27/1000
28/1000
29/1000
30/1000
31/1000
32/1000
33/1000
34/1000
35/1000
36/1000
37/1000
38/1000
39/1000
40/1000
41/1000
42/1000
43/1000
44/1000
45/1000
46/1000
47/1000
48/1000
49/1000
50/1000
51/1000
52/1000
53/1000
54/1000
55/1000
56/1000
57/1000
58/1000
59/1000
60/1000
61/1000
62/1000
63/1000
64/1000
65/1000
66/1000
67/1000
68/1000
69/1000
70/1000
71/1000
72/1000
73/1000
74/1000
75/1000
76/1000
77/1000
78/1000
79/1000
80/1000
81/1000
82/1000
83/1000
84/1000
85/1000
86/1000
87/1000
88/1000
89/1000
90/1000
91/1000
92/1000
93/1000
94/1000
95/1000
96/1000
97/1000
98/1000
99/1000
100/1000
101/1000
102/1000
103/1000
104/1000
105/1000
106/1000
107/1000
108/1000
109/1000
110/1000
111/1000
112/1000
113/1000
114/1000
115/1000
116/1000
117/1000
118/1000
119/1000
120/1000
121/1000
122/1000
123/1000
1

927/1000
928/1000
929/1000
930/1000
931/1000
932/1000
933/1000
934/1000
935/1000
936/1000
937/1000
938/1000
939/1000
940/1000
941/1000
942/1000
943/1000
944/1000
945/1000
946/1000
947/1000
948/1000
949/1000
950/1000
951/1000
952/1000
953/1000
954/1000
955/1000
956/1000
957/1000
958/1000
959/1000
960/1000
961/1000
962/1000
963/1000
964/1000
965/1000
966/1000
967/1000
968/1000
969/1000
970/1000
971/1000
972/1000
973/1000
974/1000
975/1000
976/1000
977/1000
978/1000
979/1000
980/1000
981/1000
982/1000
983/1000
984/1000
985/1000
986/1000
987/1000
988/1000
989/1000
990/1000
991/1000
992/1000
993/1000
994/1000
995/1000
996/1000
997/1000
998/1000
999/1000
1000/1000
auroc 0.8724(0.0088)
auprc 0.5363(0.0266)
minpse 0.5121(0.0222)
